In [20]:
import asyncio
import json
import re
from typing import Optional

import nest_asyncio
import pandas as pd
import tqdm
from g4f.client import AsyncClient
from googletrans import Translator
from tqdm.asyncio import tqdm_asyncio

nest_asyncio.apply()

## Refactor and save tasks dict

In [21]:
CLEAN_TAGS_R = re.compile("<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")


def remove_html_tags(text: str) -> str:
    return re.sub(CLEAN_TAGS_R, "", text)

In [22]:
def batch_list(strings: list[str], batch_size: int) -> list[list[str]]:
    return [strings[i : i + batch_size] for i in range(0, len(strings), batch_size)]


async def translate_text(text: str, src: str = "ru", dest: str = "en") -> str:
    translator = Translator()
    translation = await translator.translate(text, src=src, dest=dest)
    return translation.text


async def translate_texts_inner(
    texts: list[str], src: str = "ru", dest: str = "en"
) -> list[str]:
    translator = Translator()
    if len(texts) > 1:
        return [res.text for res in await translator.translate(texts, src=src, dest=dest)]
    res = await translator.translate(texts[0], src=src, dest=dest)
    return [res.text]


async def translate_texts(texts: list[str]) -> list[str]:
    res = await tqdm_asyncio.gather(
        *[translate_texts_inner(text_list) for text_list in batch_list(texts, 5)]
    )

    ans = []
    for r in res:
        ans.extend(r)
    return ans

In [23]:
def save_dict(tasks_dict: dict, path: str):
    with open(path, "w", encoding="utf8") as f:
        json.dump(tasks_dict, f, ensure_ascii=False, indent=4)


def build_task_str(task: dict) -> str:
    return remove_html_tags(
        f"{task['title']}\n\n"
        + f"Теги:{task['tags']}\n\n"
        + f"{task['description']}\n\n"
        + f"Формат входных данных:\n{task['inputFormat']}\n\n"
        + f"Формат выходных данных:\n{task['outputFormat']}\n\n"
        + f"Примеры\n{task['examples']}"
        + (f"\n\nПримечание:\n{task['remark']}" if task["remark"] != "" else "")
    )


def compile_tasks_dict() -> dict[str, str]:
    df = pd.read_csv("../../data/db_tasks.csv")
    raw_dict = df.fillna("").set_index("spec").to_dict(orient="index")

    return {str(spec): build_task_str(task) for spec, task in raw_dict.items()}


async def save_dicts():
    tasks_dict = compile_tasks_dict()
    save_dict(tasks_dict, "../../data/generated/db_tasks_ru.json")

    specs, values = zip(*tasks_dict.items())
    translated_values = await translate_texts(list(values))
    tasks_dict_en = dict(zip(specs, translated_values))
    save_dict(tasks_dict_en, "../../data/generated/db_tasks_en.json")


# await save_dicts()

## Load tasks dict

In [24]:
def load_dict(path: str) -> dict:
    try:
        with open(path, "r", encoding="utf8") as f:
            return json.load(f)
    except FileNotFoundError:
        return {}


tasks_dict: dict[str, str] = load_dict("../../data/generated/db_tasks_en.json")
len(tasks_dict)

553

## Generate solutions

In [25]:
AVAILABLE_MODELS = [
    "hermes-3",
    "lfm-40b",
    "dolphin-2.9",
    "lzlv-70b",
    "dolphin-2.6",
    "airoboros-70b",
    "wizardlm-2-8x22b",
    "gemini-2.0-flash-thinking",
    "llama-3.2-3b",
    "gemini-2.0",
    "gemini-2.0-flash",
    "llama-3.1-70b",
    "gemini-1.5-pro",
    "blackboxai",
    "sonar-pro",
    "claude-3.5-sonnet",
    # "gpt-4o",
    "blackboxai-pro",
    # "gpt-4",
    "evil",
    "qwen-2.5-1m",
    "glm-4",
    "qwen-2-72b",
    "gemini-1.5-flash",
]
PROMPTS = [
    # lambda task: f"Please provide python code without any external imports to solve the task below. Provide pure code without any explanations and comments. Make sure that it looks like written by human. Produce correct input/output logic. Here is the task: \n{task}",
    lambda task: f"""Write a Python solution for the following task. The code should look like it was written by an intermediate human developer—natural, readable, and practical but not overly optimized or perfect. Follow these guidelines:
Variable Names: Use descriptive but not overly long names (e.g., res instead of result or final_output_value).
Comments: Include a no comments.
Structure: Use simple functions or straightforward logic unless the problem demands complexity.
Imperfections: Small stylistic quirks are okay (e.g., mixing single/double quotes, occasional redundant logic).
Error Handling: Basic checks (e.g., for edge cases) but no excessive validation unless required.
Formatting: Use standard PEP 8 style but allow minor deviations (e.g., inconsistent spacing in one place).
Task: \n{task}.""",
]

In [26]:
def extract_python_code(res: str) -> str:
    try:
        return re.findall(r"```python(.*)```", res, re.DOTALL)[0]
    except IndexError:
        return res


async def access_llm(client: AsyncClient, model: str, prompt: str) -> Optional[str]:
    try:
        response = await client.chat.completions.create(
            model=model, messages=[{"role": "user", "content": prompt}], web_search=False
        )
        return extract_python_code(response.choices[0].message.content), model
    except Exception as _:
        return None, model

In [27]:
async def generate_solutions(
    task_dict: dict,
    prompt=PROMPTS[0],
    models: list[str] = AVAILABLE_MODELS,
) -> list:
    solutions = []

    for spec, task in tqdm.tqdm(task_dict.items()):
        client = AsyncClient()

        results = [access_llm(client, m, prompt(task)) for m in models]

        for coro in asyncio.as_completed(results):
            solution, m = await coro
            print(f"{m} Completes!")
            if solution is None or len(solution) == 0:
                continue
            solutions.append((spec, m, solution))

        await asyncio.sleep(1)

    return solutions

In [28]:
test_dict = {
    k: tasks_dict[k]
    for k in [
        "c0df7d49-26f5-451c-b44a-1e0bca60bca5",
        "4e5b21c0-e86f-4eac-82b6-1a0d00ae4199",
    ]
}

res = await generate_solutions(test_dict)

  0%|          | 0/2 [00:00<?, ?it/s]

hermes-3 Completes!
lfm-40b Completes!
gemini-1.5-pro Completes!
gemini-1.5-flash Completes!
airoboros-70b Completes!
dolphin-2.6 Completes!
dolphin-2.9 Completes!
lzlv-70b Completes!
llama-3.1-70b Completes!
wizardlm-2-8x22b Completes!
llama-3.2-3b Completes!
claude-3.5-sonnet Completes!
evil Completes!
gemini-2.0-flash-thinking Completes!
gemini-2.0 Completes!
glm-4 Completes!
blackboxai Completes!
gemini-2.0-flash Completes!
blackboxai-pro Completes!
qwen-2.5-1m Completes!
qwen-2-72b Completes!
sonar-pro Completes!


 50%|█████     | 1/2 [00:16<00:16, 16.60s/it]

hermes-3 Completes!
lfm-40b Completes!
lzlv-70b Completes!
dolphin-2.6 Completes!
dolphin-2.9 Completes!
airoboros-70b Completes!
wizardlm-2-8x22b Completes!
claude-3.5-sonnet Completes!
llama-3.2-3b Completes!
llama-3.1-70b Completes!
gemini-1.5-pro Completes!
glm-4 Completes!
gemini-1.5-flash Completes!
gemini-2.0-flash-thinking Completes!
gemini-2.0 Completes!
evil Completes!
gemini-2.0-flash Completes!
blackboxai Completes!
blackboxai-pro Completes!
qwen-2.5-1m Completes!
sonar-pro Completes!
qwen-2-72b Completes!


100%|██████████| 2/2 [00:33<00:00, 16.84s/it]


In [29]:
res

[('c0df7d49-26f5-451c-b44a-1e0bca60bca5',
  'hermes-3',
  '\ndef sequence_length():\n    length = 0\n    while True:\n        num = int(input())\n        if num == 0:\n            break\n        length += 1\n    return length\n\nprint(sequence_length())\n'),
 ('c0df7d49-26f5-451c-b44a-1e0bca60bca5',
  'lfm-40b',
  '\n# Python solution for sequence length task\ndef sequence_length():\n    count = 0\n    # Read input until 0 is encountered\n    while True:\n        num = int(input())\n        if num == 0:\n            break\n        count += 1\n    # Print the count of numbers excluding the final 0\n    print(count)\n\nsequence_length()\n'),
 ('c0df7d49-26f5-451c-b44a-1e0bca60bca5',
  'gemini-1.5-pro',
  '\ndef sequence_length():\n    count = 0\n    while True:\n        num = int(input())\n        if num == 0:\n            break\n        count += 1\n    print(count)\n\nsequence_length()\n'),
 ('c0df7d49-26f5-451c-b44a-1e0bca60bca5',
  'gemini-1.5-flash',
  '\ndef sequence_length():\n    